In [1]:
import os
import sys

import pandas as pd

os.chdir("../")

In [60]:
sys.path.append("./")
from animaloc_improved.tools.massage_results import conf_matrix_to_pct, massage_metrics

# Entrenamientos de Faster-RCNN

## Entrenamiento con backbones Resnet-50 y Resnet-101

Entrenamiento se realizó sobre parches de 512 x 512 con overlap de 160.

Hiperparámetros vienen de `configs/train/faster_rcnn_herdnet_trainer.yaml` y el número de épocas se establece en la línea de comando:


```python
{
    # del .yaml:
    trainable_backbone_layers: 4,  # abreviado en nombre de archivo tbl4
    batch_size: 8,
    optimizer: "adam",
    learning_rate: 1e-5,
    weight_decay: 0.0005,
    warmup_iters: 100,
    # especificados por línea de comando:
    epochs: 100
}
```

### Entrenamiento con backbone resnet-50
```bash
dvc -s train-faster-rcnn-hn-trainer

# Commando ejecutado por dvc
python oficial_herdnet/tools/train.py  --config-path $PWD/configs/
   train=faster_rcnn_herdnet_trainer
   train.training_settings.work_dir=data/models/faster-rcnn/resnet50-100-epochs-tbl4
   train.training_settings.epochs=100
```

### Entrenamiento con backbone resnet-101
```bash
dvc repro -s train-faster-rcnn-resnet-101

# comando de python
python oficial_herdnet/tools/train.py  --config-path $PWD/configs/
train=faster_rcnn_herdnet_trainer
train.wandb_mode=offline
train.wandb_run=faster-rcnn-resnet-101
train.model.kwargs.architecture=resnet101
train.training_settings.work_dir=data/models/faster-rcnn/resnet101-100-epochs-tbl4
train.training_settings.epochs=100

```

In [99]:
!ls -lh --block-size=M data/models/faster-rcnn/resnet50-100-epochs-tbl4 \
    | awk '{print $5, $6, $7, $9}'

   
1M Oct 21 20251020_training.log
1M Oct 20 20251020_validation.log
474M Oct 21 best_model-by-epoch-70.pth
474M Oct 21 best_model.pth
474M Oct 21 latest_model.pth


In [100]:
!ls -lh --block-size=M data/models/faster-rcnn/resnet101-100-epochs-tbl4 \
    | awk '{print $5, $6, $7, $9}'

   
1M Oct 26 20251024_training.log
1M Oct 25 20251024_validation.log
691M Oct 26 best_model.pth
691M Oct 26 latest_model.pth


#### Descripción de los modelo con backbones resnet-50 y resnet-101

```bash
# resnet-50:
python animaloc_improved/tools/describe_model.py --config-path=$PWD/configs
      test=faster_rcnn
      test.device_name="cpu"
      test.model.kwargs.architecture=resnet50
      test.model.kwargs.trainable_backbone_layers=4
      test.model.pth_file=data/models/faster-rcnn/resnet50-100-epochs-tbl4/best_model.pth
      +test.group_depth=2
      +test.results_file=data/model_descs/parameters-frcnn-resnet-50.csv

# resnet-101:
python tools/describe_model.py --config-path=$PWD/configs
    test=faster_rcnn
    test.device_name="cpu"
    test.model.kwargs.architecture=resnet101
    test.model.kwargs.trainable_backbone_layers=4
    test.model.pth_file=data/models/faster-rcnn/resnet101-100-epochs-tbl4/best_model.pth
    +test.group_depth=2
    +test.results_file=data/model_descs/parameters-frcnn-resnet-101.csv
```

In [7]:
resnet50_desc = pd.read_csv("data/model_descs/parameters-frcnn-resnet-50.csv")
resnet50_desc

,group,trainable,num_params_MM
0,model.backbone,False,0.009408
1,model.backbone,True,26.789888
2,model.roi_heads,True,13.931555
3,model.rpn,True,0.593935


In [86]:
resnet101_desc = pd.read_csv("data/model_descs/parameters-frcnn-resnet-101.csv")
resnet101_desc

,group,trainable,num_params_MM
0,model.backbone,False,0.009408
1,model.backbone,True,45.729792
2,model.roi_heads,True,13.931555
3,model.rpn,True,0.593935


In [9]:
# resnet50: Total de parámetros [MM]
round(sum(resnet50_desc["num_params_MM"] * resnet50_desc["trainable"]), 1)

41.3

In [88]:
# resnet101: Total de parámetros [MM]
round(sum(resnet101_desc["num_params_MM"] * resnet101_desc["trainable"]), 1)

60.3

#### Evaluaciones

Evaluaciines sobre sobre imágenes de test a tamaño completo.

Importante aquí se usó un threshold de detección de 0.3, que es relativamente bajo.

```bash
dvc repro -s test-faster-rcnn-resnet50-full:

# Como las imágenes tienen tamaños diversos hay que poner test.datasetbatch_size=1
  python oficial_herdnet/tools/test.py  --config-path $PWD/configs/
  test=faster_rcnn
  test.device_name="cuda"
  test.wandb_mode="online"
  test.dataset.batch_size=1
  test.wandb_run=test-frcnn-resnet50
  test.model.pth_file=data/models/faster-rcnn/resnet50-100-epochs-tbl4/best_model.pth
  test.model.kwargs.architecture=resnet50
  test.evaluator.threshold=0.3
  test.dataset.csv_file=data/gt-preprocessed/csv/test_big_size_A_B_E_K_WH_WB-fixed-header.csv
  test.dataset.root_dir=data/test/
  test.results_dir=data/test_results/frcnn-resnet50-full-imgs
```


In [98]:
# resultados:
!ls -lh --block-size=k  data/test_results/frcnn-resnet50-full-imgs \
   | awk '{print $5, $6, $7, $9}'

   
1K Oct 26 confusion_matrix.csv
953K Oct 26 detections.csv
2K Oct 26 metrics_results.csv
4K Oct 26 plots


In [91]:
# Resnet-50
conf_matrix_to_pct(pd.read_csv("data/test_results/frcnn-resnet50-full-imgs/confusion_matrix.csv"))
# Filas: clase real, Columnas: clase predicha

,Alcelaphinae,Buffalo,Kob,Warthog,Waterbuck,Elephant
Especie,,,,,,
Alcelaphinae,95.8,0.7,2.0,0.9,0.5,0.0
Buffalo,1.7,82.5,0.7,14.9,0.3,0.0
Kob,21.7,0.0,77.7,0.5,0.2,0.0
Warthog,0.0,0.0,2.6,92.1,5.3,0.0
Waterbuck,45.2,12.9,12.9,3.2,25.8,0.0
Elephant,0.2,0.2,0.0,0.0,0.0,99.5


In [92]:
# Resnet-101
conf_matrix_to_pct(pd.read_csv("data/test_results/frcnn-resnet101-full-imgs/confusion_matrix.csv"))

,Alcelaphinae,Buffalo,Kob,Warthog,Waterbuck,Elephant
Especie,,,,,,
Alcelaphinae,94.3,1.3,2.4,0.7,1.3,0.0
Buffalo,2.1,79.9,0.0,16.7,1.4,0.0
Kob,18.2,0.0,81.1,0.7,0.0,0.0
Warthog,0.0,0.0,3.2,93.5,3.2,0.0
Waterbuck,12.5,3.1,0.0,3.1,81.2,0.0
Elephant,0.0,0.2,0.0,0.0,0.0,99.8


In [82]:
metrics = pd.read_csv("data/test_results/frcnn-resnet50-full-imgs/metrics_results.csv")
massage_metrics(metrics)

,Especie,N,Rec.[%],Prec.[%],F1[%],Conf.[%],MAE,AP [%]
0,Alcelaphinae,675,78.4,48.6,60.0,4.2,3.4,56.8
1,Buffalo,349,71.6,54.5,61.9,17.5,2.6,55.9
2,Kob,477,72.1,65.9,68.9,22.3,1.6,59.0
3,Warthog,74,47.3,7.7,13.3,7.9,3.3,18.9
4,Waterbuck,36,22.2,18.6,20.3,74.2,2.1,8.2
5,Elephant,688,61.6,10.7,18.2,0.5,30.4,27.6
6,binary,2299,78.0,27.4,40.6,0.0,16.5,40.5


In [93]:
metrics = pd.read_csv("data/test_results/frcnn-resnet101-full-imgs/metrics_results.csv")
massage_metrics(metrics)

,Especie,N,Rec.[%],Prec.[%],F1[%],Conf.[%],MAE,AP [%]
0,Alcelaphinae,675,76.3,55.4,64.2,5.7,2.7,55.1
1,Buffalo,349,65.9,57.5,61.4,20.1,2.5,48.7
2,Kob,477,74.8,58.2,65.5,18.9,1.8,58.1
3,Warthog,74,39.2,9.5,15.3,6.5,3.2,17.4
4,Waterbuck,36,72.2,40.6,52.0,18.8,1.3,54.3
5,Elephant,688,64.7,13.3,22.0,0.2,25.4,32.3
6,binary,2299,77.6,31.5,44.8,0.0,13.2,42.0


In [84]:
!dvc repro -s faster-rcnn-resnet50-full-to-excel

Running stage 'faster-rcnn-resnet50-full-to-excel':
> python animaloc_improved/tools/results_to_excel.py data/test_results/frcnn-resnet50-full-imgs -o data/test_results/frcnn-resnet50-full-imgs-xlsx
Confusion matrix [%]
| Especie      |   Alcelaphinae |   Buffalo |   Kob |   Warthog |   Waterbuck |   Elephant |
|:-------------|---------------:|----------:|------:|----------:|------------:|-----------:|
| Alcelaphinae |           95.8 |       0.7 |   2   |       0.9 |         0.5 |        0   |
| Buffalo      |            1.7 |      82.5 |   0.7 |      14.9 |         0.3 |        0   |
| Kob          |           21.7 |       0   |  77.7 |       0.5 |         0.2 |        0   |
| Warthog      |            0   |       0   |   2.6 |      92.1 |         5.3 |        0   |
| Waterbuck    |           45.2 |      12.9 |  12.9 |       3.2 |        25.8 |        0   |
| Elephant     |            0.2 |       0.2 |   0   |       0   |         0   |       99.5 |
Metrics:
|    | Especie      |    N 

In [94]:
!dvc repro -s faster-rcnn-resnet101-full-to-excel

Running stage 'faster-rcnn-resnet101-full-to-excel':                            
> python animaloc_improved/tools/results_to_excel.py data/test_results/frcnn-resnet101-full-imgs -o data/test_results/frcnn-resnet101-full-imgs-xlsx
Confusion matrix [%]
| Especie      |   Alcelaphinae |   Buffalo |   Kob |   Warthog |   Waterbuck |   Elephant |
|:-------------|---------------:|----------:|------:|----------:|------------:|-----------:|
| Alcelaphinae |           94.3 |       1.3 |   2.4 |       0.7 |         1.3 |        0   |
| Buffalo      |            2.1 |      79.9 |   0   |      16.7 |         1.4 |        0   |
| Kob          |           18.2 |       0   |  81.1 |       0.7 |         0   |        0   |
| Warthog      |            0   |       0   |   3.2 |      93.5 |         3.2 |        0   |
| Waterbuck    |           12.5 |       3.1 |   0   |       3.1 |        81.2 |        0   |
| Elephant     |            0   |       0.2 |   0   |       0   |         0   |       99.8 |
Metri